In [ ]:
FEATURES = 5
CLIENT_SIZE = 2                         # number of clients
CLIENTS_BATCH_SIZES = [100, 150]        # number of each client's data samples


n = FEATURES                            # each client has data with n features

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Set a fixed seed for reproducibility
SEED = 321123
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Synthetic Data Preparation

## Client Data

- **Feature Matrix (`X`)** `m x n`: Created using `np.random.randn(m, n)`, which generates values from a standard normal distribution (mean=0, variance=1).
- **True Coefficients** (weights `w`) and **Intercept** (bias term `b`): Randomly sampled from a normal distribution to define the linear relationship.
- **Target Vector** (`y = X @ w + b`): Computed as a linear combination of features and coefficients, plus Gaussian noise to simulate real-world data variability.

In [ ]:
def generate_linear_regression_data(m: int, n: int, mean=0, std=1, noise_std=0.5):
    """
    Generates synthetic data for linear regression.

    Parameters:
    - m: Number of samples.
    - n: Number of features per sample.
    - noise_std (float): Standard deviation of the Gaussian noise added to y.

    Returns:
    - X (np.ndarray): Feature matrix of shape (m, n).
    - y (np.ndarray): Target vector of shape (m,).
    """

    # Generate feature matrix X from a standard normal distribution
    X = np.random.normal(mean, std, (m, n))

    # Generate true coefficients (weights) and intercept (bias)
    w = np.random.randn(n, 1)  # true coefficients
    b = np.random.randn()      # intercept term

    y = X.dot(w) + b           # y = X @ w + b         y.shape = (m, 1)

    # Add Gaussian noise to the target values
    y += np.random.normal(0, noise_std, (m, 1))

    return X, y

## Weight Matrix

We need to generate a matrix `W` `n+1+1 x n+2`, the columns of which represent `n+2` (why `n+2` – to make `W` a **square** matrix) randomly generated weight vectors, each of which contains `n+1` elements w0, w1, w2, etc. and

the additional row of `1`'s will count for **the free term coefficient** of a gradient function $ \frac{\partial L(\textbf{w}, \;\textbf{x})}{\partial w_j} $, where $ j ∈ [0, .., n] $.

Also, to guarantee that `W` is **invertible**, we will make it [Diagonally Dominant Matrix](https://stackoverflow.com/questions/73426718/generating-invertible-matrices-in-numpy-tensorflow) over columns as


"A **strictly diagonally dominant matrix** (or an irreducibly diagonally dominant matrix) is **non-singular**."

In [ ]:
def generate_weight_matrix(n):
    W = np.random.rand(n+1, n+2).astype(np.float32)
    W = np.concatenate([W, np.ones((1, n+2))], axis=0)

    diag = np.sum(np.abs(W), axis=0) + 1
    np.fill_diagonal(W, diag)
    W[n+1, n+1] = 1         # the row of 1's was also affected, so reassigning a value of 1 again.

    return W

# Federated Learning Functions

## Client Calculate Gradients

`L` `n+1 x n+2` – matrix with all the gradient vector updates for the corresponding weight vectors from `W`.


In [ ]:
def client_calculate_gradients(W, X, y):
    batch_size = X.shape[0]

    # Add an intercept column
    X = np.hstack([np.ones((batch_size, 1)), X])

    # Remove the last row of one's from W
    W = W[:-1, :]

    # Make Y matrix out of n+2 copies of y to count for n+2 random sets of weights
    Y = np.hstack([y]*(n+2))

    # Calculate the gradient dL/dw
    L = (1/batch_size) * X.T@(X @ W - Y)

    return L

# Simulation

## Client Data

In [ ]:
X = []
y = []

In [ ]:
# Generate
for batch_size in CLIENTS_BATCH_SIZES:
    Xi, yi = generate_linear_regression_data(batch_size, n, noise_std=0.5)
    X.append(Xi)
    y.append(yi)

In [ ]:
# Or load from .csv
for i in range(CLIENT_SIZE):
    X.append(np.genfromtxt(f'X_{i}.csv', delimiter=','))
    y.append(np.genfromtxt(f'y_{i}.csv', delimiter=','))

## Weight matrix W

In [ ]:
# Generate
W = generate_weight_matrix(n)

In [ ]:
# Or load from .csv
W = np.genfromtxt('W.csv', delimiter=',')

In [ ]:
# Check the determinant of W
np.linalg.det(W)

9489.335952151043

## Gradient Calculation For Each Client

In [ ]:
L = []

for i in range(CLIENT_SIZE):
    # Calculate gradient Lᵢ
    L.append(client_calculate_gradients(W, X[i], y[i]))

## Sum the gradient update matrices `Lᵢ`

`Lᵢ` is the matrix received from `i`th client, also `n+1 x n+2`.

In [ ]:
L = np.sum(L, axis=0)

## Find the matrix C

$ C \times W = L $  
$ ^{n+1 \times n+2} $ $ ^{n+1+1 \times n+2}$ $^{=}$ $ ^{n+1 \times n+2} $

$ C = L \times W^{-1} $  
$ ^{n+1 \times n+2} $ $^{=}$ $ ^{n+1 \times n+2} $ $ ^{n+2 \times n+2}$

In [ ]:
C = L @ np.linalg.inv(W)

## Find the optimal set of weights

Find `w_opt` such that `C @ w_opt = 0`

In [ ]:
A = C[:, :-1]
b = C[:, -1] * -1
w_opt = np.linalg.solve(A, b)    # [w0, w1, w2, etc.]

In [ ]:
w_opt

array([ 0.71957472,  1.31259721,  0.10155391, -0.50612878,  0.01317545,
        0.78649842])